In [ ]:
import os
import pickle
import face_recognition

# 📂 Definir la carpeta donde se guardan los embeddings
embeddings_dir = "embeddings/"
os.makedirs(embeddings_dir, exist_ok=True)

# 📂 Ruta del dataset de imágenes
dataset_path = "dataset/"

# 🔄 Recorrer cada carpeta dentro de `dataset/` (cada carpeta es una persona)
for person_name in os.listdir(dataset_path):
    person_folder = os.path.join(dataset_path, person_name)
    
    if not os.path.isdir(person_folder):
        continue  # Ignorar archivos que no sean carpetas
    
    # 📌 Verificar si la persona ya tiene un archivo `.pkl`
    person_embeddings_path = os.path.join(embeddings_dir, f"{person_name}.pkl")
    if os.path.exists(person_embeddings_path):
        print(f"⚠️ {person_name} ya tiene embeddings guardados, se omite.")
        continue  # Saltar esta persona y pasar a la siguiente

    print(f"📸 Procesando imágenes de {person_name}...")

    person_encodings = []

    # 🔎 Leer cada imagen dentro de la carpeta
    for filename in os.listdir(person_folder):
        image_path = os.path.join(person_folder, filename)
        
        try:
            image = face_recognition.load_image_file(image_path)

            # 📌 Detectar rostro en la imagen (usando HOG)
            face_locations = face_recognition.face_locations(image, model="hog")
            face_encodings = face_recognition.face_encodings(image, known_face_locations=face_locations)

            if face_encodings:
                person_encodings.append(face_encodings[0])  # Guardar el embedding de la persona
                print(f"✅ Embedding generado para {filename}")
            else:
                print(f"⚠️ No se detectó rostro en {filename}, se omite.")

        except Exception as e:
            print(f"❌ Error procesando {filename}: {e}")

    # 💾 Guardar los embeddings en un archivo .pkl separado para cada persona
    if person_encodings:
        with open(person_embeddings_path, "wb") as f:
            pickle.dump({"encodings": person_encodings, "name": person_name}, f)
        print(f"\n✅ Embeddings guardados en {person_embeddings_path}")

print("\n🚀 Procesamiento completado!")